In [1]:
import pandas as pd

# merging 4

In [10]:
quotes4 = pd.DataFrame(
    {
        "time": [
            pd.Timestamp("2016-05-25 13:30:23"),
            pd.Timestamp("2016-05-25 13:30:23"),
        ],
        "ticker": [
               "MSFT",
               "MSFT",
           ],
           "bid": [ 51.95, 51.97],
           "ask": [51.96, 51.98]
    }
)

In [11]:
trades4 = pd.DataFrame(
       {
           "time": [
               pd.Timestamp("2016-05-25 13:30:23"),
               pd.Timestamp("2016-05-25 13:30:38")
           ],
           "ticker": ["MSFT", "MSFT"],
           "price": [51.95, 51.95],
           "quantity": [75, 155]
       }
   )

In [12]:
quotes4

,time,ticker,bid,ask
0,2016-05-25 13:30:23,MSFT,51.95,51.96
1,2016-05-25 13:30:23,MSFT,51.97,51.98


In [13]:
trades4

,time,ticker,price,quantity
0,2016-05-25 13:30:23,MSFT,51.95,75
1,2016-05-25 13:30:38,MSFT,51.95,155


In [14]:
pd.merge_asof(trades4,quotes4, on="time", by="ticker", tolerance=pd.Timedelta("1s"))

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:23,MSFT,51.95,75,51.97,51.98
1,2016-05-25 13:30:38,MSFT,51.95,155,NaN,NaN


In [15]:
# trades4['time'].dt.microsecond

In [16]:
#gets duplicates when in left dataframe "trades" both values within tolerance e.g. 23, 24, see under #3
#but when right dataframe 23,23 then only gets one (the second one(?))

#==>found out here 2 things
#1)duplicates need to be removed  (when left dataframe has one value that would match more than once)
#2)additional matches will not be found when more than one value in the right df would match with one value in left)

In [17]:
#will dollar amount be aggregated when there are several loops?

# Merging 5

In [2]:
quotes5 = pd.DataFrame(
    {
        "time": [
            pd.Timestamp("2016-05-25 13:30:23"),
            pd.Timestamp("2016-05-25 13:30:23"),
            pd.Timestamp("2016-05-25 13:30:23"),
            pd.Timestamp("2016-05-25 13:30:23")
        ],
        "ticker": [
               "MSFT",
               "MSFT",
               "MSFT",
               "MSFT"
           ],
           "bid": [ 51.95, 51.97,51.99,52.01],
           "ask": [51.96, 51.98,52.00,52.02],
        "time2":[23,23,23,23]
    }
)

In [3]:
trades5 = pd.DataFrame(
       {
           "time": [
               pd.Timestamp("2016-05-25 13:30:23"),
               pd.Timestamp("2016-05-25 13:30:24"),
               pd.Timestamp("2016-05-25 13:30:25"),
               pd.Timestamp("2016-05-25 13:30:35")
           ],
           "ticker": ["MSFT", "MSFT","MSFT","MSFT"],
           "price": [51.95, 11.11,33.33,99.99],
           "quantity": [75, 155, 190,270],
           "time2": [23,24,25,35]
       }
   )

In [6]:
merged=pd.merge_asof(trades5,quotes5, on="time", by="ticker",suffixes=['_tr','_qu'], tolerance=pd.Timedelta("1s"))
merged #duplicates need to be removed  (left df has 2 values that match with the same right df value)--> re-use 2nd value 
       #in left df after removing 1st matched values in both left and right dfs

,time,ticker,price,quantity,time2_tr,bid,ask,time2_qu
0,2016-05-25 13:30:23,MSFT,51.95,75,23,52.01,52.02,23.0
1,2016-05-25 13:30:24,MSFT,11.11,155,24,52.01,52.02,23.0
2,2016-05-25 13:30:25,MSFT,33.33,190,25,NaN,NaN,NaN
3,2016-05-25 13:30:35,MSFT,99.99,270,35,NaN,NaN,NaN


In [56]:
mask=merged['bid'].isna()
mask

0    False
1    False
2     True
3     True
Name: bid, dtype: bool

In [57]:
unmatched=merged[mask].copy()
unmatched

,time,ticker,price,quantity,time2_tr,bid,ask,time2_qu
2,2016-05-25 13:30:25,MSFT,33.33,190,25,NaN,NaN,NaN
3,2016-05-25 13:30:35,MSFT,99.99,270,35,NaN,NaN,NaN


In [58]:
matched=merged[~mask].copy()
matched

,time,ticker,price,quantity,time2_tr,bid,ask,time2_qu
0,2016-05-25 13:30:23,MSFT,51.95,75,23,52.01,52.02,23.0
1,2016-05-25 13:30:24,MSFT,11.11,155,24,52.01,52.02,23.0


In [59]:
macthed_num_dup=matched.shape[0]
macthed_num_dup

2

In [63]:
matched=(matched.assign(DIFF=lambda x: (x['time2_tr']-x['time2_qu'])))#/
                        #.apply(lambda x: x.n).abs()).sort_values(by='DIFF',ascending=True).drop_duplicates(subset="bid"))
matched

,time,ticker,price,quantity,time2_tr,bid,ask,time2_qu,DIFF
0,2016-05-25 13:30:23,MSFT,51.95,75,23,52.01,52.02,23.0,0.0
1,2016-05-25 13:30:24,MSFT,11.11,155,24,52.01,52.02,23.0,1.0


In [65]:
# matched=matched.apply(lambda x: x.n).abs()
matched=matched.sort_values(by='DIFF',ascending=True).drop_duplicates(subset="bid")
matched

,time,ticker,price,quantity,time2_tr,bid,ask,time2_qu,DIFF
0,2016-05-25 13:30:23,MSFT,51.95,75,23,52.01,52.02,23.0,0.0


In [66]:
macthed_num_dup=matched.shape[0]
macthed_num_dup

1